In [1]:
import pandas as pd
import sqlite3
conn = sqlite3.connect(r'data/flights.db')
# https://www.dataquest.io/blog/python-pandas-databases/

# tables - airports, airlines, routes


# return all tables in database
# query ="""SELECT name from sqlite_master where type= "table";"""


In [20]:
conn = sqlite3.connect(r'data/flights.db')

query = """INSERT INTO airlines ('name', 'alias', 'iata', 'icao', 'callsign', 'country', 'active')
           VALUES (?, ?, ?, ?, ?, ?, ?)"""
params = ("my_ariline_1", "MA1", "A", "B", "MA1", "EU", "Y",)

cursor = conn.cursor()
cursor.execute(query, params)
conn.commit()
cursor.close()

TypeError: 'autocommit' is an invalid keyword argument for this function

In [18]:

query ="""SELECT * 
            FROM airlines"""
df = pd.read_sql(query, 
                conn,)
df.tail()

,index,id,name,alias,iata,icao,callsign,country,active
6045,6045.0,19831,Fly Europa,None,ER,RWW,None,Spain,Y
6046,6046.0,19834,FlyPortugal,None,PO,FPT,FlyPortugal,Portugal,Y
6047,6047.0,19845,FTI Fluggesellschaft,None,None,FTI,None,Germany,N
6048,NaN,None,my_ariline_1,MA1,A,B,MA1,EU,Y
6049,NaN,None,None,None,A,B,MA1,EU,Y


In [2]:

query ="""SELECT * 
            FROM airports
            WHERE country = '{}' 
            LIMIT {}""".format("United States", 200)
df = pd.read_sql(query, 
                conn,)
df[["latitude","longitude"]] = df[["latitude","longitude"]].astype(float)
df

,index,id,name,city,country,code,icao,latitude,longitude,altitude,offset,dst,timezone
0,318,6891,Putnam County Airport,Greencastle,United States,4I7,\N,39.633556,-86.813806,842,-5,U,America/New_York
1,1104,6890,Dowagiac Municipal Airport,Dowagiac,United States,C91,\N,41.992934,-86.128012,748,-5,U,America/New_York
2,1121,6889,Cambridge Municipal Airport,Cambridge,United States,CDI,\N,39.975028,-81.577583,799,-5,U,America/New_York
3,1470,6885,Door County Cherryland Airport,Sturgeon Bay,United States,SUE,\N,44.843667,-87.421556,725,-6,U,America/Chicago
4,1507,6884,Shoestring Aviation Airfield,Stewartstown,United States,0P2,\N,39.794824,-76.647191,1000,-5,U,America/New_York
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,3470,3569,Palmer Muni,Palmer,United States,PAQ,PAAQ,61.594914,-149.088711,242,-9,A,America/Anchorage
196,3471,3570,Pittsburgh Intl,Pittsburgh,United States,PIT,KPIT,40.491467,-80.232872,1204,-5,A,America/New_York
197,3472,3571,Wiley Post Will Rogers Mem,Barrow,United States,BRW,PABR,71.285446,-156.766003,44,-9,A,America/Anchorage
198,3473,3572,Ellington Fld,Houston,United States,EFD,KEFD,29.607333,-95.158750,32,-6,A,America/Chicago


In [89]:
fig = px.scatter_mapbox(df, 
                        lat="latitude",
                        lon="longitude",
                        zoom=3, 
                        height=300, 
                        hover_name="name", 
                        hoverinfo="text",
                        hover_data=["city", "code"])
fig.update_layout(
    mapbox_style="white-bg",
    mapbox_layers=[
        {
            "below": 'traces',
            "sourcetype": "raster",
            "sourceattribution": "United States Geological Survey",
            "source": [
                "https://basemap.nationalmap.gov/arcgis/rest/services/USGSImageryOnly/MapServer/tile/{z}/{y}/{x}"
            ]
        }
      ])
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

TypeError: scatter_mapbox() got an unexpected keyword argument 'hoverinfo'

In [90]:
import plotly.express as px
df = px.data.gapminder().query("year == 2007")
fig = px.line_geo(df, locations="iso_alpha",
                  color="continent", # "continent" is one of the columns of gapminder
                  projection="orthographic")
fig.show()

In [91]:
import plotly.graph_objects as go
import pandas as pd

df_airports = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_february_us_airport_traffic.csv')
df_airports.head()

df_flight_paths = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_february_aa_flight_paths.csv')
df_flight_paths.head()

fig = go.Figure()

fig.add_trace(go.Scattergeo(
    locationmode = 'USA-states',
    lon = df_airports['long'],
    lat = df_airports['lat'],
    hoverinfo = 'text',
    text = df_airports['airport'],
    mode = 'markers',
    marker = dict(
        size = 2,
        color = 'rgb(255, 0, 0)',
        line = dict(
            width = 3,
            color = 'rgba(68, 68, 68, 0)'
        )
    )))

lons = []
lats = []
import numpy as np
lons = np.empty(3 * len(df_flight_paths))
lons[::3] = df_flight_paths['start_lon']
lons[1::3] = df_flight_paths['end_lon']
lons[2::3] = None
lats = np.empty(3 * len(df_flight_paths))
lats[::3] = df_flight_paths['start_lat']
lats[1::3] = df_flight_paths['end_lat']
lats[2::3] = None

fig.add_trace(
    go.Scattergeo(
        locationmode = 'USA-states',
        lon = lons,
        lat = lats,
        mode = 'lines',
        line = dict(width = 1,color = 'red'),
        opacity = 0.5
    )
)

fig.update_layout(
    title_text = 'Feb. 2011 American Airline flight paths<br>(Hover for airport names)',
    showlegend = False,
    geo = go.layout.Geo(
        scope = 'north america',
        projection_type = 'azimuthal equal area',
        showland = True,
        landcolor = 'rgb(243, 243, 243)',
        countrycolor = 'rgb(204, 204, 204)',
    ),
    height=700,
)

fig.show()

# Different ways to add parameters

In [68]:
query = """select "Timestamp","Value" from "MyTable" 
                      where "Timestamp" BETWEEN %(dstart)s AND %(dfinish)s"""
params = {"dstart":datetime(2014,6,24,16,0),
            "dfinish":datetime(2014,6,24,17,0)}
df = psql.read_sql(query,
                   db,
                   params=params,
                   index_col=['Timestamp'])

NameError: name 'datetime' is not defined

In [ ]:
query = """select "Timestamp","Value" from "MyTable" 
                      where "Timestamp" BETWEEN %s AND %s"""
params = (datetime(2014,6,24,16,0), datetime(2014,6,24,17,0))
df = psql.read_sql(query,
                   db,params=params,
                   index_col=['Timestamp'])